In [89]:
import pandas as pd

df = pd.read_csv('data_1.csv')

df.head()

,category,text,clean_text
0,arts,rob delaney vir das galen hopper samson kayo g...,rob da galen hopper samson kayo guz khan nick ...
1,arts,andris nelsons conducts a joint concert of the...,joint concert boston symphony orchestra visiti...
2,arts,warner music group has brought on sherry tan t...,warner music group brought sherry tan head mus...
3,arts,adele will explore what she s been going throu...,explore going new album set explore going new ...
4,arts,you are using an older browser version please ...,older browser version please use version best ...


In [87]:
df["category"].unique()

array(['arts', 'crime', 'disaster', 'economy', 'education',
       'environmental', 'health', 'humanInterest', 'labour', 'lifestyle',
       'other', 'politics', 'religion', 'science', 'social', 'sport',
       'unrest', 'weather'], dtype=object)

In [92]:
from collections import Counter
import pandas as pd

def count_common_words(df):
    category_word_counts = {}

    for category in df["category"].unique():
        # Get all text for this category
        category_text = " ".join(df[df["category"] == category]["text"])

        # Tokenize and count word frequencies
        words = category_text.split()
        word_counts = Counter(words)

        # Store the word frequency dictionary
        category_word_counts[category] = dict(word_counts)

    return category_word_counts

# Get word counts per category
word_counts_by_category = count_common_words(df)

# Display the top 10 most common words for each category
for category, word_counts in word_counts_by_category.items():
    print(f"Category: {category}")
    print(dict(Counter(word_counts).most_common(25)))  # Show top 10 words
    print("-" * 40)

Category: arts
{'the': 8198, 'and': 4248, 'to': 3802, 'of': 3665, 'a': 3466, 'in': 2918, 's': 1772, 'for': 1676, 'on': 1417, 'is': 1367, 'with': 1277, 'that': 1239, 'it': 1065, 'at': 963, 'as': 853, 'i': 845, 'this': 775, 'was': 754, 'you': 738, 'be': 701, 'from': 681, 'by': 678, 'are': 676, 'will': 665, 'we': 652}
----------------------------------------
Category: crime
{'the': 7911, 'to': 3867, 'a': 3668, 'of': 3541, 'and': 3415, 'in': 3162, 'that': 1466, 's': 1420, 'on': 1329, 'for': 1288, 'was': 1172, 'is': 989, 'with': 967, 'said': 935, 'by': 836, 'he': 826, 'at': 803, 'as': 737, 'it': 686, 'his': 676, 'from': 639, 'police': 608, 'an': 576, 'has': 529, 'be': 506}
----------------------------------------
Category: disaster
{'the': 7803, 'to': 3562, 'and': 3540, 'of': 3281, 'a': 3163, 'in': 2834, 'on': 1324, 'for': 1155, 's': 1136, 'that': 1134, 'is': 1063, 'was': 909, 'said': 894, 'with': 840, 'at': 825, 'from': 734, 'it': 734, 'as': 665, 'by': 607, 'are': 605, 'fire': 579, 'be': 5

In [102]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from nltk import sent_tokenize
import nltk
from collections import Counter
import nltk
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


import copy

weighted_dict= {}

# 1) Original weighted dictionary
# weighted_dict = {
#     "arts": {
#         "artist": 1.0,
#         "exhibition": 0.9,
#         "gallery": 0.9,
#         "sculpture": 0.8,
#         "performance": 0.8,
#         "museum": 0.7,
#         "creative": 0.6,
#         "painting": 0.7,
#     },
#     "crime": {
#         "crime": 1.0,
#         "theft": 0.9,
#         "murder": 0.95,
#         "police": 0.8,
#         "arrest": 0.85,
#         "fraud": 0.75,
#         "investigation": 0.7,
#         "law": 0.6,
#     },
#     "disaster": {
#         "earthquake": 0.95,
#         "flood": 0.9,
#         "hurricane": 0.9,
#         "disaster": 1.0,
#         "emergency": 0.8,
#         "rescue": 0.75,
#         "casualty": 0.7,
#         "evacuation": 0.7,
#         "fire": 0.9,
#     },
#     "economy": {
#         "gdp": 0.9,
#         "inflation": 0.95,
#         "market": 0.85,
#         "trade": 0.8,
#         "stock": 0.75,
#         "recession": 0.9,
#         "currency": 0.7,
#         "deficit": 0.7,
#     },
#     "education": {
#         "school": 0.9,
#         "student": 0.95,
#         "teacher": 0.85,
#         "university": 0.8,
#         "curriculum": 0.7,
#         "tuition": 0.7,
#         "degree": 0.6,
#         "exam": 0.6,
#     },
#     "environmental": {
#         "pollution": 0.95,
#         "climate": 0.9,
#         "recycling": 0.8,
#         "sustainability": 0.85,
#         "carbon": 0.9,
#         "emissions": 0.85,
#         "wildlife": 0.7,
#         "conservation": 0.75,
#     },
#     "health": {
#         "hospital": 0.9,
#         "disease": 0.95,
#         "vaccine": 0.85,
#         "patient": 0.8,
#         "medical": 0.85,
#         "pandemic": 1.0,
#         "treatment": 0.7,
#         "mental": 0.6,
#     },
#     "humaninterest": {
#         "story": 0.9,
#         "award": 1.0,
#         "community": 0.8,
#         "charity": 0.75,
#         "volunteer": 0.7,
#         "personal": 0.85,
#         "inspiration": 0.7,
#         "family": 0.6,
#         "achievement": 0.65,
#     },
#     "labour": {
#         "union": 0.9,
#         "employment": 0.8,
#         "strike": 0.85,
#         "wage": 0.9,
#         "contract": 0.7,
#         "workers": 0.95,
#         "work": 0.9,
#         "job": 0.85,
#         "security": 0.85,
#         "benefits": 0.6,
#         "overtime": 0.6,
#     },
#     "lifestyle": {
#         "fashion": 0.9,
#         "travel": 0.85,
#         "diet": 0.8,
#         "hobby": 0.7,
#         "wellness": 0.75,
#         "luxury": 0.6,
#         "trend": 0.7,
#         "fitness": 0.6,
#         "home": 0.85,
#     },
#     "politics": {
#         "government": 0.95,
#         "election": 0.9,
#         "policy": 0.85,
#         "diplomacy": 0.8,
#         "senate": 0.75,
#         "vote": 0.9,
#         "democracy": 0.7,
#         "law": 0.6,
#         "president": 0.9,
#         "state": 0.8,
#     },
#     "religion": {
#         "faith": 0.9,
#         "temple": 0.8,
#         "prayer": 0.85,
#         "ritual": 0.75,
#         "church": 0.8,
#         "islam": 0.7,
#         "christianity": 0.7,
#         "worship": 0.7,
#         "god": 0.85,
#         "spiritual": 0.8,
#     },
#     "science": {
#         "research": 0.95,
#         "experiment": 0.9,
#         "technology": 0.85,
#         "data": 0.8,
#         "discovery": 0.85,
#         "physics": 0.75,
#         "biology": 0.7,
#         "innovation": 0.7,
#     },
#     "social": {
#         "equality": 0.9,
#         "welfare": 0.85,
#         "activism": 0.8,
#         "diversity": 0.75,
#         "rights": 0.9,
#         "justice": 0.7,
#         "community": 0.6,
#         "protest": 0.65,
#         "society": 0.8,
#     },
#     "sport": {
#         "athlete": 0.95,
#         "match": 0.9,
#         "tournament": 0.85,
#         "score": 0.8,
#         "league": 0.75,
#         "olympics": 0.7,
#         "training": 0.6,
#         "championship": 0.8,
#     },
#     "unrest": {
#         "protest": 0.9,
#         "riot": 0.95,
#         "clash": 0.85,
#         "violence": 0.9,
#         "tension": 0.8,
#         "conflict": 0.75,
#         "demonstration": 0.7,
#         "angry": 0.6,
#     },
#     "weather": {
#         "forecast": 0.9,
#         "storm": 0.95,
#         "temperature": 0.8,
#         "rain": 0.75,
#         "drought": 0.7,
#         "sunny": 0.6,
#         "climate": 0.6,
#         "hurricane": 0.7,
#     },
# }

# 2) New words from your dataset (per category)
new_keywords = {
    'arts': [
        'music','show','series','park','best','million','event','back','work','state',
        'way','may','get','group','take','world','community','season','good','school'
    ],
    'crime': [
        'police','court','law','state','case','county','old','found','may','hearing',
        'death','federal','prison','told','justice','man','right','officer','according','president'
    ],
    'disaster': [
        'fire','water','police','air','state','rescue','city','may','area','county',
        'home','near','according','news','around','could','back','national','help','road'
    ],
    'economy': [
        'company','may','per','market','million','price','tax','billion','business','government',
        'state','since','industry','group','percent','use','get','made','work','according'
    ],
    'education': [
        'school','university','education','high','learning','college','state','student','class',
        'community','work','program','support','help','board','government','president','covid','public','national'
    ],
    'environmental': [
        'energy','park','waste','water','power','market','state','company','north','management',
        'solar','group','renewable','county','world','wildlife','city','wind','river','industry'
    ],
    'health': [
        'health','covid','care','market','medical','pandemic','hospital','state','may','vaccine',
        'public','food','disease','get','help','company','global','well','virus','work'
    ],
    'humaninterest': [
        'award','size','get','act','home','back','winning','best','statement','texas',
        'team','may','good','world','well','ownership','beneficial','plant','second','top'
    ],
    'labour': [
        'work','health','security','working','pandemic','retirement','may','covid','could','get',
        'state','make','job','business','home','help','social','market','city','week'
    ],
    'lifestyle': [
        'home','travel','get','beauty','bridge','make','summer','may','way','life',
        'work','back','million','come','food','covid','want','pandemic','best','business'
    ],
    'other': [
        'state','market','game','covid','government','may','could','rub','wednesday','home',
        'high','city','get','school','pandemic','health','make','season','back','team'
    ],
    'politics': [
        'government','state','president','minister','data','security','public','city','international','support',
        'tuesday','news','united','may','covid','national','pandemic','help','house','health'
    ],
    'religion': [
        'church','life','school','family','may','community','world','way','come','work',
        'home','covid','old','get','back','spiritual','god','state','well','city'
    ],
    'science': [
        'market','research','space','report','global','data','work','company','study','industry',
        'may','science','medical','could','high','design','engineering','covid','well','use'
    ],
    'social': [
        'health','care','social','work','government','covid','may','baby','state','abortion',
        'support','home','get','life','pandemic','according','community','make','city','help'
    ],
    'sport': [
        'team','game','season','back','world','get','match','second','sport','four',
        'league','win','best','final','play','tournament','coach','right','open','championship'
    ],
    'unrest': [
        'israel','police','war','government','city','violence','shooting','israeli','military','may',
        'wednesday','state','gun','country','president','old','come','security','world','home'
    ],
    'weather': [
        'climate','change','heat','weather','water','high','carbon','could','global','national',
        'state','may','risk','area','service','according','world','friday','rain','report'
    ],
}



# 3) Create a copy so we don't modify original directly
updated_dict = copy.deepcopy(weighted_dict)

# 4) Define a default weight for new words
DEFAULT_WEIGHT = 0.6

# 5) Merge logic
for category, words in new_keywords.items():
    # If this category doesn't exist in the dictionary, create it
    if category not in updated_dict:
        updated_dict[category] = {}
    
    for w in words:
        # Add the word only if it isn't already in the dictionary
        if w not in updated_dict[category]:
            updated_dict[category][w] = DEFAULT_WEIGHT

# 'updated_dict' is now the merged result.


# x = df["clean_text"]
# y = df["category"]
# # Train the Random Forest classifier
# X_train, X_test, y_train, y_test = train_test_split(
#     x, y, test_size=0.2, random_state=42
# )
# 
# vectorizer = TfidfVectorizer()
# X_train_vectorized = vectorizer.fit_transform(X_train)
# X_test_vectorized = vectorizer.transform(X_test)
# 
# rf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf.fit(X_train_vectorized, y_train)
# 
# y_pred = rf.predict(X_test_vectorized)
# 
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)
# 
# def dictionary_score(sentence, weighted_dict):
#     """
#     Computes a score for each class based on the presence of weighted keywords.
#     """
#     words = sentence.lower().split()
#     scores = {cls: 0 for cls in weighted_dict}
#     for cls in weighted_dict:
#         for word in words:
#             scores[cls] += weighted_dict[cls].get(word, 0)
#     return scores
# 
# def classifier_predict(sentence):
#     X_sentence = vectorizer.transform([sentence])
#     probas = rf.predict_proba(X_sentence)[0]
#     return dict(zip(rf.classes_, probas))
# 
# def combine_predictions(clf_pred, dict_scores, alpha=0.5):
#     """
#     Combine classifier probabilities and dictionary scores.
#     The parameter alpha weights the classifier output.
#     """
#     combined = {}
#     for cls in clf_pred:
#         combined[cls] = alpha * clf_pred[cls] + (1 - alpha) * dict_scores.get(cls, 0)
#     return combined
# 
# def classify_text(text, alpha=0.5):
#     """
#     Tokenize the text into sentences, combine classifier and dictionary scores,
#     and return the final predicted class based on majority vote.
#     """
#     sentences = sent_tokenize(text)
#     sentence_labels = []
#     
#     for sentence in sentences:
#         clf_pred = classifier_predict(sentence)
#         dict_scores = dictionary_score(sentence, weighted_dict)
#         combined_scores = combine_predictions(clf_pred, dict_scores, alpha)
#         # Select the class with the highest combined score for the sentence
#         sentence_label = max(combined_scores, key=combined_scores.get)
#         sentence_labels.append(sentence_label)
#     
#     # Final classification by taking the most common class (majority vote)
#     final_label = Counter(sentence_labels).most_common(1)[0][0]
#     return final_label
# 
# # Example usage:
# text_row = ("The gallery showcased a variety of paintings and sculptures. "
#             "A recent experiment in the laboratory produced groundbreaking results.")
# final_prediction = classify_text(text_row, alpha=0.6)
# print("Final predicted class:", final_prediction)

In [103]:
def dictionary_score(sentence, weighted_dict):
    """
    Computes a score for each class based on the presence of weighted keywords.
    """
    words = sentence.lower().split()
    scores = {cls: 0 for cls in weighted_dict}
    for cls in weighted_dict:
        for word in words:
            scores[cls] += weighted_dict[cls].get(word, 0)
    return scores

def classifier_predict(sentence):
    """
    Uses the trained Random Forest and TF-IDF vectorizer to get class probabilities.
    """
    X_sentence = vectorizer.transform([sentence])
    probas = rf.predict_proba(X_sentence)[0]
    return dict(zip(rf.classes_, probas))

def combine_predictions(clf_pred, dict_scores, alpha=0.5):
    """
    Combine classifier probabilities and dictionary scores.
    The parameter alpha weights the classifier output.
    """
    combined = {}
    for cls in clf_pred:
        combined[cls] = alpha * clf_pred[cls] + (1 - alpha) * dict_scores.get(cls, 0)
    return combined

def classify_text(text, alpha=0.5):
    """
    Tokenize the text into sentences, combine classifier and dictionary scores,
    and return the final predicted class based on majority vote.
    """
    sentences = sent_tokenize(text)
    sentence_labels = []
    
    for sentence in sentences:
        clf_pred = classifier_predict(sentence)
        dict_scores = dictionary_score(sentence, weighted_dict)
        combined_scores = combine_predictions(clf_pred, dict_scores, alpha)
        
        # Select the class with the highest combined score for the sentence
        sentence_label = max(combined_scores, key=combined_scores.get)
        sentence_labels.append(sentence_label)
    
    # Final classification by taking the most common class (majority vote)
    final_label = Counter(sentence_labels).most_common(1)[0][0]
    return final_label

# 
# df_bad = pd.read_csv("ethanCleaned.csv")


from tqdm import tqdm
tqdm.pandas()  # Enable progress_apply on pandas objects
import numpy as np

df["predicted_category"] = df["clean_text"].progress_apply(lambda x: classify_text(x, alpha=0.5))

df.head()


# df_bad["predicted_category"] = np.where(
#     (df_bad["matches"] != 1.0) | df_bad["matches"].isna(),
#     df_bad["clean_text"].astype(str).progress_apply(lambda x: classify_text(x, alpha=0.5) if pd.notna(x) and isinstance(x, str) else np.nan),
#     np.nan  # Assign NaN where matches == 1.0
# )
# 
# df_bad.head()


100%|██████████| 5292/5292 [03:32<00:00, 24.86it/s]


,category,text,clean_text,predicted_category
0,arts,rob delaney vir das galen hopper samson kayo g...,rob da galen hopper samson kayo guz khan nick ...,health
1,arts,andris nelsons conducts a joint concert of the...,nelson conduct joint concert boston symphony o...,labour
2,arts,warner music group has brought on sherry tan t...,warner music group brought sherry tan head mus...,politics
3,arts,adele will explore what she s been going throu...,explore going new album set explore going new ...,humaninterest
4,arts,you are using an older browser version please ...,older browser version please use version best ...,politics


In [59]:
# # Update 'category' only where 'predicted_category' is not NaN
# df_bad["category"] = np.where(df_bad["predicted_category"].notna(), df_bad["predicted_category"], df_bad["category"])
# 
# # Create 'total_category' to reflect the final combined categories
# df["total_category"] = df_bad["category"]
# df.head()

,category,text,clean_text,words_removed,text2,total_category
0,arts,you are using an older browser version. please...,you are an older browser version . please use ...,50,older browser version please use version best ...,arts
1,arts,on 31 march two of classical music s most acco...,on 31 march two of classical music s most acco...,156,march two classical music accomplished well kn...,arts
2,arts,bpt after a year of being locked away at home ...,after a year of being locked away at home the ...,98,year locked away home world eager reopen exper...,arts
3,arts,pilot uninjured plane hit sandbar while landin...,pilot uninjured plane hit while landing . a fl...,22,pilot uninjured plane hit landing float plane ...,humaninterest
4,arts,colleen distin photo by facebook toronto sun ....,colleen photo by sun . a lost wallet been retu...,47,colleen photo sun lost wallet returned owner c...,arts


In [104]:
from nltk.corpus import stopwords, words
from nltk import WordNetLemmatizer, word_tokenize
import re
# Download NLTK resources (run once)
import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
english_vocab = set(w.lower() for w in words.words())

def clean_text(text):
    """
    Cleans and preprocesses the input text, removing
    URLs, HTML tags, punctuation, stopwords, and any word
    that is not in a large English vocabulary set.
    """
    # Step 1: Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Step 2: Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Step 3: Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Step 4: Convert to lowercase
    text = text.lower()
    
    # Step 5: Tokenize text into words
    tokens = word_tokenize(text)
    
    # Step 6: Remove stopwords and short tokens
    tokens = [word for word in tokens if word not in stop_words and len(word) >= 3]
    
    # Step 7: Lemmatize words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Step 8: Keep only English words (based on nltk.corpus.words)
    tokens = [word for word in tokens if word in english_vocab]

    # Step 9: Join tokens back into a single string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Example usage
df["clean_text"] = df["text"].apply(clean_text)

df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/karennurlybekov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/karennurlybekov/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/karennurlybekov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/karennurlybekov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,category,text,clean_text,predicted_category
0,arts,rob delaney vir das galen hopper samson kayo g...,rob da galen hopper samson kayo guz khan nick ...,health
1,arts,andris nelsons conducts a joint concert of the...,nelson conduct joint concert boston symphony o...,labour
2,arts,warner music group has brought on sherry tan t...,warner music group brought sherry tan head mus...,politics
3,arts,adele will explore what she s been going throu...,explore going new album set explore going new ...,humaninterest
4,arts,you are using an older browser version please ...,older browser version please use version best ...,politics


In [105]:
# import numpy as np
# 
# df["total_category"] = np.where(df["category"] == "other", "other", df["predicted_category"])

df['predicted_category'].value_counts()

    

predicted_category
politics         2631
education         482
health            380
sport             324
crime             295
humaninterest     229
environmental     162
labour            158
economy           138
weather           127
lifestyle         117
science           110
disaster           73
religion           45
arts               15
unrest              4
social              2
Name: count, dtype: int64

In [99]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Example DataFrame
# df has columns "text2" and "total_category"
x = df["text"]
y = df["predicted_category"]

# 1. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

# 2. TF-IDF vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

param_distributions = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [5, 10, 15, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    # 'max_features': ['sqrt', 'log2', None],  # Optionally include this
    "bootstrap": [True, False], 
    "criterion": ["gini", "entropy"],  
    # You can also consider tuning class_weight, e.g.:
    # "class_weight": ["balanced", "balanced_subsample", None]
}



# 3. Random Forest model
rf = RandomForestClassifier(
    n_estimators=300, 
    random_state=42, 
    class_weight="balanced", 
)
rf.fit(X_train_vectorized, y_train)

# 4. Get the usual predictions
y_pred = rf.predict(X_test_vectorized)

# 5. Get predicted probabilities
y_proba = rf.predict_proba(X_test_vectorized)

# 6. Define a threshold for "not sure"
threshold = 0.2

# 7. Create a new set of predictions, switching to "other" if max probability < threshold
y_pred_with_other = []
for i, probs in enumerate(y_proba):
    if np.max(probs) < threshold:
        y_pred_with_other.append("other")
    else:
        # Keep the original predicted class
        y_pred_with_other.append(y_pred[i])

# 8. Evaluate
# Note: If you include "other" in predictions but it is not in y_test, 
#       classification_report will show 0 for precision/recall/f1 of "other"
#       and metrics for original classes will not change.
accuracy_with_other = accuracy_score(y_test, y_pred_with_other)
print("Accuracy (with 'other'):", accuracy_with_other)

print("\nClassification Report (with 'other'):")
print(classification_report(y_test, y_pred_with_other))

Accuracy (with 'other'): 0.5051935788479698

Classification Report (with 'other'):
               precision    recall  f1-score   support

         arts       1.00      0.04      0.08        23
        crime       0.68      0.74      0.71        65
     disaster       0.75      0.12      0.21        48
      economy       0.81      0.32      0.46        41
    education       0.81      0.79      0.80        97
environmental       0.77      0.54      0.64        50
       health       0.71      0.63      0.67       103
humaninterest       0.67      0.36      0.47       106
       labour       0.77      0.29      0.42        94
    lifestyle       0.88      0.11      0.20        62
        other       0.00      0.00      0.00         0
     politics       0.44      0.83      0.58       172
     religion       1.00      0.28      0.43        29
      science       0.79      0.24      0.37        46
       social       0.00      0.00      0.00        14
        sport       0.61      0.72  

/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", l

In [107]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Example DataFrame: df with columns "text2" and "total_category"
x = df["text"]
y = df["predicted_category"]

# 1. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

# 2. TF-IDF vectorization
vectorizer = TfidfVectorizer(
    analyzer='word',
    token_pattern=r'(?u)\b\w\w+\b',
    lowercase=True,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8,
    max_features=50000,
    sublinear_tf=True,
    norm='l2',
    smooth_idf=True
)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# 3. Define parameter distributions for RandomizedSearchCV
param_distributions = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [5, 10, 15, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False],
    "criterion": ["gini", "entropy"],
    "class_weight": ["balanced", None, "balanced_subsample"],
}

# 4. Base RandomForest model
rf = RandomForestClassifier(random_state=42)

# 5. RandomizedSearchCV setup
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_distributions,
    n_iter=30,
    cv=3,
    scoring='accuracy',
    random_state=42,
    verbose=1,
    n_jobs=-1
)

# 6. Fit random search on training data
random_search.fit(X_train_vectorized, y_train)

# 7. Get the best model and hyperparameters
best_rf = random_search.best_estimator_
print("Best Hyperparameters found by RandomizedSearchCV:")
print(random_search.best_params_)

# 8. Predict on test data (standard prediction, no custom threshold)
y_pred = best_rf.predict(X_test_vectorized)

# 9. Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits


/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


Best Hyperparameters found by RandomizedSearchCV:
{'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': None, 'criterion': 'gini', 'class_weight': 'balanced_subsample', 'bootstrap': True}

Accuracy: 0.8083097261567517

Classification Report:
               precision    recall  f1-score   support

         arts       1.00      1.00      1.00         1
        crime       0.75      0.92      0.83        63
     disaster       0.57      0.67      0.62         6
      economy       0.84      0.80      0.82        20
    education       0.80      0.92      0.86        93
environmental       0.64      0.90      0.75        31
       health       0.69      0.89      0.78        66
humaninterest       0.47      0.23      0.31        30
       labour       0.59      0.54      0.57        24
    lifestyle       0.50      0.33      0.40        15
     politics       0.89      0.80      0.84       598
     religion       0.60      0.60      0.60         5
      science

In [29]:
from sentence_transformers import SentenceTransformer

model_name = "all-mpnet-base-v2"
embedder = SentenceTransformer(model_name)

corpus = df["clean_text"].tolist()
embeddings = embedder.encode(corpus, show_progress_bar=True)

Batches:   0%|          | 0/166 [00:00<?, ?it/s]

In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

X = embeddings
y = df["predicted_category"].values

# Split data into train/test (or train/val/test if you prefer)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 5, 10],
    "max_features": ['sqrt', 'log2'],
    "class_weight": ['balanced_subsample', 'balanced', None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    # Add more params if desired, e.g. "min_samples_split": [2, 5]
}

rf_clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    rf_clf,
    param_grid=param_grid,
    cv=2,               # 3-fold cross validation
    scoring="accuracy", # or "accuracy", "f1_weighted", etc.
    n_jobs=-1,          # utilize all CPU cores
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best params:", grid_search.best_params_)
print("Best CV score (accuracy):", grid_search.best_score_)


best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nTEST SET PERFORMANCE:")
print(classification_report(y_test, y_pred, zero_division=0))

KeyError: 'predicted_category'

In [83]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load Data
df_bad = pd.read_csv("ethanCleaned.csv")

# Drop rows where "matches" or "clean_text" or "category" is NaN
df_bad = df_bad.dropna(subset=["matches", "clean_text", "category"])

# Extract features (X) and target labels (y)
X = df_bad["clean_text"]
y = df_bad["category"]

# 1. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. TF-IDF vectorization
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_train_vectorized, y_train = ros.fit_resample(X_train_vectorized, y_train)

# 3. Random Forest model
rf = RandomForestClassifier(
    n_estimators=300, 
    random_state=42, 
    class_weight="balanced_subsample",
)
rf.fit(X_train_vectorized, y_train)

# 4. Get the usual predictions
y_pred = rf.predict(X_test_vectorized)

# 5. Get predicted probabilities
y_proba = rf.predict_proba(X_test_vectorized)


# 8. Evaluate
accuracy_with_other = accuracy_score(y_test, y_pred_with_other)
print("Accuracy (with 'other'):", accuracy_with_other)

print("\nClassification Report (with 'other'):")
print(classification_report(y_test, y_pred_with_other))


Accuracy (with 'other'): 0.21676646706586827

Classification Report (with 'other'):
               precision    recall  f1-score   support

         arts       1.00      0.05      0.10        56
        crime       0.79      0.29      0.42        52
     disaster       0.82      0.20      0.32        45
      economy       1.00      0.07      0.12        46
    education       0.68      0.50      0.58        52
environmental       1.00      0.21      0.34        58
       health       0.53      0.18      0.27        45
humaninterest       1.00      0.02      0.05        43
       labour       0.67      0.05      0.09        43
    lifestyle       0.00      0.00      0.00        50
        other       0.05      0.72      0.08        39
     politics       1.00      0.02      0.04        49
     religion       0.75      0.18      0.29        33
      science       0.42      0.11      0.17        47
       social       0.00      0.00      0.00        43
        sport       0.50      0.50 

/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/karennurlybekov/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()